In [7]:
"""
Docs for spotify track mining:

https://spotipy.readthedocs.io/en/2.9.0/
"""

import requests
import sys
from spotipy.oauth2 import SpotifyClientCredentials
import numpy as np
import random
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import json
import time
import urllib
from sklearn.preprocessing import MultiLabelBinarizer

import sys
import spotipy
import spotipy.util as util

In [8]:
# shows a user's playlists (need to be authenticated via oauth)
client_id = "fc38b7cf17894dffb6f84dede9b1385c"
client_secret = "a94b446d21ef4b64a44c58cd14e27ed7"
username = "elska.santiska"

# Setup the credentials
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)

# Make the connection
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [18]:
"""
Code adapted from here:

https://morioh.com/p/31b8a607b2b0
"""
def tracks_list(tracks):
    l = []
    for i, item in enumerate(tracks['items']):
        track = item['track']
        l.append(track['id'])
    return l
     
def getTrackIDs(user, playlist_id):
    ids = []
    playlist = sp.user_playlist(user, playlist_id)
    """
    for item in playlist['tracks']['items']:
    track = item['track']
    ids.append(track['id'])
    results = sp.playlist(playlist['id'], fields="tracks,next")
    tracks = results['tracks']
    """
    print()
    print(playlist['name'])
    print ('  total tracks', playlist['tracks']['total'])
    results = sp.playlist(playlist['id'], fields="tracks,next")
    tracks = results['tracks']
    l = tracks_list(tracks)
    
    ids = ids + l
    show_tracks(tracks)
    
    while tracks['next']:
        tracks = sp.next(tracks)
        l = tracks_list(tracks)
        show_tracks(tracks)
        ids = ids + l
    #ids = [val for sublist in ids for val in sublist]

    return ids

ids = getTrackIDs(username, '4ZGPXnMqwrDxDra3u5Ifyo')

print(len(ids))


Disco Tracks
  total tracks 347
   0               Earth, Wind & Fire September
   1                      Wild Cherry Play That Funky Music
   2                         Bee Gees Stayin' Alive - From "Saturday Night Fever" Soundtrack
   3                    Stevie Wonder Superstition
   4                  Kool & The Gang Get Down On It
   5           KC & The Sunshine Band That's the Way (I Like It) - 2004 Remaster
   6                       Diana Ross I'm Coming Out
   7  James Brown & The Famous Flames I Got You (I Feel Good)
   8                  Kool & The Gang Celebration - Single Version
   9                            Wham! Wake Me Up Before You Go-Go
   10                     Carl Douglas Kung Fu Fighting
   11                 Michael Sembello Maniac - BOF Flashdance
   12                     Dire Straits Walk Of Life
   13                       Chaka Khan What Cha' Gonna Do for Me
   14           KC & The Sunshine Band Give It Up
   15                   Modern Talking Cheri Ch

In [19]:
"""
Function from here:

https://morioh.com/p/31b8a607b2b0
"""

def getTrackFeatures(id):
    meta = sp.track(id)
    features = sp.audio_features(id)

    # meta
    name = meta['name']
    album = meta['album']['name']
    artist = meta['album']['artists'][0]['name']
    release_date = meta['album']['release_date']
    length = meta['duration_ms']
    popularity = meta['popularity']

    # features
    acousticness = features[0]['acousticness']
    danceability = features[0]['danceability']
    energy = features[0]['energy']
    instrumentalness = features[0]['instrumentalness']
    liveness = features[0]['liveness']
    loudness = features[0]['loudness']
    speechiness = features[0]['speechiness']
    tempo = features[0]['tempo']
    time_signature = features[0]['time_signature']

    track = [name, album, artist, release_date, length, popularity, danceability, acousticness, danceability, energy, instrumentalness, liveness, loudness, speechiness, tempo, time_signature]
    #track = [name, album, artist, release_date]
    
    return track

In [20]:
# loop over track ids 
tracks = []
for i in range(len(ids)):
    time.sleep(.5)
    track = getTrackFeatures(ids[i])
    tracks.append(track)

# create dataset
df = pd.DataFrame(tracks, columns = ['name', 'album', 'artist', 'release_date', 'length', 'popularity', 'danceability', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature'])
df.to_csv("spotify.csv", sep = ',')

In [3]:
def show_tracks(tracks):
    for i, item in enumerate(tracks['items']):
        track = item['track']
        print("   %d %32.32s %s" % (i, track['artists'][0]['name'], track['name']))
        
# Note that my playlists are the aggregation of multiple playlists of the same genre
playlists = sp.user_playlists(username)
for playlist in playlists['items']:
    if playlist['owner']['id'] == username:
        print()
        print(playlist['name'])
        print ('  total tracks', playlist['tracks']['total'])
        results = sp.playlist(playlist['id'], fields="tracks,next")
        tracks = results['tracks']
        show_tracks(tracks)
        while tracks['next']:
            tracks = sp.next(tracks)
            show_tracks(tracks)


Disco Tracks
  total tracks 347
   0               Earth, Wind & Fire September
   1                      Wild Cherry Play That Funky Music
   2                         Bee Gees Stayin' Alive - From "Saturday Night Fever" Soundtrack
   3                    Stevie Wonder Superstition
   4                  Kool & The Gang Get Down On It
   5           KC & The Sunshine Band That's the Way (I Like It) - 2004 Remaster
   6                       Diana Ross I'm Coming Out
   7  James Brown & The Famous Flames I Got You (I Feel Good)
   8                  Kool & The Gang Celebration - Single Version
   9                            Wham! Wake Me Up Before You Go-Go
   10                     Carl Douglas Kung Fu Fighting
   11                 Michael Sembello Maniac - BOF Flashdance
   12                     Dire Straits Walk Of Life
   13                       Chaka Khan What Cha' Gonna Do for Me
   14           KC & The Sunshine Band Give It Up
   15                   Modern Talking Cheri Ch

   0                  Waylon Jennings I Ain't Living Long Like This
   1                    George Strait If You're Thinking You Want A Stranger (There's One Coming Home)
   2                    John Anderson Swingin'
   3               Hank Williams, Jr. All My Rowdy Friends Are Coming Over Tonight
   4                     Travis Tritt Country Club
   5                    Eddie Rabbitt Drivin' My Life Away - 2009 Remaster
   6                       Shenandoah Two Dozen Roses
   7                    Dwight Yoakam Guitars, Cadillacs
   8                    T.G. Sheppard War Is Hell - Re-Recorded
   9                  Waylon Jennings Theme from "The Dukes of Hazzard" (Good Ol' Boys)
   10                    George Strait All My Ex's Live In Texas
   11                    Merle Haggard I Think I'll Just Stay Here And Drink
   12               Hank Williams, Jr. All My Rowdy Friends (Have Settled Down)
   13                    Ronnie Milsap Prisoner of the Highway
   14                    

   0                        Mac Davis Stop And Smell The Roses
   1                      Tom T. Hall The Year That Clayton Delaney Died
   2                     George Jones The Grand Tour
   3                    Merle Haggard It's Not Love (But It's Not Bad)
   4                     Don Williams Tulsa Time
   5                    Eddie Rabbitt Drinkin' My Baby (Off My Mind) - 2009 Remaster
   6                     The Kendalls Heaven's Just A Sin Away
   7                    Freddy Fender Wasted Days And Wasted Nights
   8                   Emmylou Harris Two More Bottles of Wine - 2003 Remaster
   9                  David Allan Coe You Never Even Called Me By My Name
   10                    Tammy Wynette You and Me
   11                      John Denver Thank God I'm a Country Boy - Live
   12                      Ray Stevens Misty
   13                     Loretta Lynn Coal Miner's Daughter
   14                       Billy Swan I Can Help
   15                    Ronnie Milsap Day

   0                     The Wreckers Leave the Pieces
   1                Montgomery Gentry Something to Be Proud Of
   2                   Tracy Lawrence Find out Who Your Friends Are
   3                    Blake Shelton Honey Bee
   4                        Lee Brice Hard To Love
   5                     Dustin Lynch Where It's At
   6                    Rascal Flatts What Hurts The Most
   7                      Josh Turner Why Don't We Just Dance
   8             Florida Georgia Line Round Here
   9                  Gretchen Wilson Redneck Woman
   10                       Luke Bryan Someone Else Calling You Baby
   11                     Alan Jackson Where I Come From
   12                       The Chicks Cowboy Take Me Away
   13                 Carrie Underwood Cowboy Casanova
   14                     Craig Morgan That's What I Love About Sunday
   15                 Billy Currington Don't It
   16                  Thompson Square Are You Gonna Kiss Me Or Not
   17          

   0                    Wanda Jackson Right Or Wrong
   1                       Don Gibson Sea of Heartbreak
   2                      Claude King Wolverton Mountain
   3                       Bobby Bare 500 Miles Away from Home
   4                     Connie Smith Once a Day
   5                   Porter Wagoner We'll Get Ahead Someday
   6                     Loretta Lynn You Ain't Woman Enough - Single Version
   7                 Jeannie C. Riley Harper Valley P.T.A.
   8                  Ernest Ashworth Talk Back Trembling Lips
   9                      Patsy Cline Leavin' On Your Mind - Single Version
   10                      Johnny Cash Jackson (with June Carter Cash)
   11                      Eddy Arnold Make the World Go Away
   12                        Ray Price Don't You Ever Get Tired of Hurting Me
   13                     Wynn Stewart It's Such A Pretty World Today
   14                   Porter Wagoner The Last Thing On My Mind
   15                     Floyd Cramer

   0                     Freddie King Going Down
   1                        B.B. King Chains And Things
   2                        Otis Rush Double Trouble
   3                     Howlin' Wolf Smokestack Lightnin'
   4                      Earl Hooker Two Bugs And A Roach
   5                       Slim Harpo I'm A King Bee
   6                     Muddy Waters Rollin' Stone
   7               Bobby "Blue" Bland Ain't No Love In The Heart Of The City - Single Version
   8                   Luther Allison Bad News Is Coming
   9               Professor Longhair Big Chief - Remastered
   10                 Hound Dog Taylor Give Me Back My Wig
   11                        Taj Mahal Leaving Trunk
   12                      Koko Taylor Wang Dang Doodle
   13                  John Lee Hooker Boogie Chillen
   14                        Magic Sam All Your Love
   15             Allman Brothers Band Trouble No More
   16                      Albert King Crosscut Saw - Mono Mix
   17         

   0               Joanne Shaw Taylor Blackest Day
   1                      Colin James More Than You Needed
   2                  John Mayer Trio Gravity - Live In Concert
   3                     Coco Montoya Nothing But Love
   4              Ronnie Baker Brooks Times Have Changed (feat. Al Kapone)
   5                     Van Morrison Too Many Myths
   6                   Laith Al-Saadi Can't Keep Going
   7                    Joe Bonamassa Ain't No Way
   8                Shemekia Copeland Long As I Can See The Light
   9              Henrik Freischlader Desert Love
   10   Ronnie Earl & The Broadcasters Blues for Henry
   11                       John Mayer Gravity
   12                   Jimmy Thackery Lickin' Gravy
   13                     Coco Montoya Am I Losing You
   14                      Robben Ford Ain't Got Nothin' but the Blues
   15                            Meena Just as I Am
   16   Little Charlie & The Nightcats If This Is Love
   17                     Freddie

   0               Queen Sylvia Embry Going Upstairs
   1                    Jimmy Dawkins Off Business
   2          Scotty And The Rib Tips Careless With Our Love
   3                   Betty Fox Band Down
   4                   Betty Fox Band Slow Burn
   5                     Eric Steckel It's My Own Fault
   6                        Wes Jeans Drownin' On Dry Land
   7                     Billy Branch Blue And Lonesome
   8                       Chris Cain Late Night City Blues
   9                        Buddy Guy DJ Play My Blues
   10                   Vaneese Thomas I Tried
   11                     Junior Wells Key To The Highway
   12      Christone "Kingfish" Ingram Empty Promises
   13             Backtrack Blues Band Times Is Hard
   14                      Ana Popovic Slow Dance
   15                         Mandalyn Wrecked
   16                      Lloyd Jones A True Love Never Dies
   17                     Howlin' Wolf Smokestack Lightnin'
   18                      

   0                       Cary Morin When the Levee Breaks (feat. Jean Jacques Milteau)
   1               Stevie Ray Vaughan Life By The Drop
   2                       Doc Watson Deep River Blues
   3                        Eric Bibb Brazos River Blues
   4                      Lazer Lloyd Backstreets
   5                  The Cold Stares Break My Fall
   6                 Kelly Joe Phelps Lead Me On
   7                        Eric Bibb With My Maker I Am One
   8                       Steve Hill Going Down the Road Feeling Bad
   9                    Seasick Steve Don't Ask Me (Bonus Track)
   10                   Ryan Montbleau Ships in the Night
   11                      Brian Casey Oh Now
   12                    John Moreland You Don't Care for Me Enough to Cry
   13                       Rory Block Walking Blues
   14                 Joe Louis Walker Going To Canada
   15                      J.B. Lenoir Alabama Blues
   16                  The Cold Stares Under His Command


   0                        Dr. Alban Let The Beat Go On - Short
   1                        Too $hort Blow the Whistle
   2                        Dr. Alban I Feel The Music
   3                        Dr. Alban Reggae Gone Ragga
   4                        Dr. Alban No Coke - 7" Mix
   5                         Lil Keed Hibachi (feat. Young Thug)
   6                         Lil Keed Kiss Em Peace (feat. Young Thug)
   7                         Lil Keed Intro
   8                Martyn Bootyspoon Ice Cream Mane
   9                         Madreblu Certamente
   10                          Lil Wil My Dougie
   11                             T.I. Live Your Life
   12                             T.I. Whatever You Like
   13                         Hot Boys Respect My Mind
   14                         Hot Boys Block Burner
   15                      Soulja Slim From What I Was Told
   16                       Snoop Dogg Gangsta Luv
   17                     Rocío Dúrcal Déjame Vivir (w